# SparkContext - numărul de noduri _worker_ și evaluarea _lazy_

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3344186e23272e2b762a2d194f7958ad190a8839a78bbba228dd268dfa035068
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


## Verificarea impactului numărului de _workers_
La inițializarea unui `SparkContext` putem specifica numărul de noduri _worker_. În general, este recomandat să avem un nod _worker_/_core_ al mașinii, însă acest număr poate fi diferit. În următoarele celule de cod vom examina impactul numărului de _workers_ pe câteva operații paralelizate.

In [2]:
from time import time
from pyspark import SparkContext

In [ ]:
for j in range(1,5):
    sc= SparkContext(master = "local[%d]"%(j))
    print(sc.defaultParallelism)
    t0=time()
    for i in range(10):
        sc.parallelize([1,2]*10000).reduce(lambda x,y:x+y)
    print(f"{j} executors, time = {time()-t0}")
    sc.stop()

1
1 executors, time = 2.6631362438201904
2
2 executors, time = 5.029542446136475
3
3 executors, time = 5.02923583984375
4
4 executors, time = 7.652313470840454


In [3]:
cat/proc/cpuinfo


processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

In [4]:
!nproc


2


In [ ]:
#sc.defaultParallelism
#sc.stop()
#t1=time()
time()-t1

#### Analizăm următorul rezultat: <br>1 executors, time = 1.6259338855743408<br>2 executors, time = 0.8395042419433594<br>3 executors, time = 0.7692611217498779<br>4 executors, time = 0.7232298851013184
#### Observăm că durează aproape dublu pentru 1 _worker_, iar apoi valorile sunt apropiate pentru 2, 3, 4 _workers_. Aceasta are loc deoarece codul este rulat pe o  mașină virtuală Linux ce utilizează doar 2 core-uri ale mașinii gazdă. În cazul în care codul de mai sus este rulat pe o mașină cu 4 core-uri, se vor observa îmbunătățiri până la 4 core-uri, urmate de o aplatizare. <br> Utilizarea mai multor workers/core nu este benefică deoarece, în acel caz, nu se face decât o schimbare de context ce nu accelerează calculul paralel.

## Ideea evaluării de tip _lazy_ : amânarea execuției până la momentul la care aceasta este cu adevărat necesară

### Creați un RDD cu 1 million de elemente

In [5]:
sc = SparkContext(master="local[2]")

In [6]:
%%time
rdd1 = sc.parallelize(range(1000000))

CPU times: user 10.2 ms, sys: 171 µs, total: 10.3 ms
Wall time: 856 ms


### Creați o funcție denumită `taketime` ce calculează valoarea cosinus a primelor 100 de valori întregi, iar la final returnează cos(x), unde x a fost primit ca parametru.

In [7]:
from math import cos

def taketime(x):
  [cos(j) for j in range(100)]
  return cos(x)

### Cât timp durează execuția lui `taketime`?

In [8]:
%%time
taketime(2)

CPU times: user 60 µs, sys: 0 ns, total: 60 µs
Wall time: 66.3 µs


-0.4161468365471424

### Aplicați operația `map` utilizând funcția anterioară

In [15]:
%%time
interim = rdd1.map(lambda x: taketime(x))

CPU times: user 46 µs, sys: 6 µs, total: 52 µs
Wall time: 57.5 µs


#### Se observă că fiecare execuție a lui _taketime_ durează similar, iar operația _map_ pe un RDD cu 1000000 elemente are o durată asemănătoare. <br><br>Acest lucru are loc datorită evaluării _lazy_, adică nimic nu este calculat în celula precedentă, ci are loc doar o planificare a execuției. Variabila `interim` nu pointează către o structură de date, ci către un plan de execuție exprimat prin intermediul unui graf de dependențe. Acest graf de dependențe definește modul în care un RDD este calculat pornind de la un altul.

### Putem vizualiza graful de dependențe utilizând metoda `toDebugString`

In [16]:
print(interim.toDebugString().decode())

(2) PythonRDD[1] at RDD at PythonRDD.scala:53 []
 |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289 []


![](https://raw.githubusercontent.com/tirthajyoti/Spark-with-Python/master/Images/RDD_dependency_graph.PNG)

### Execuția efectivă prin metoda `reduce`

In [17]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

output = -0.2887054679684353
CPU times: user 101 ms, sys: 13.1 ms, total: 114 ms
Wall time: 16 s


In [ ]:
1000000*31e-6

31.0

#### Remarcăm timpul relativ scurt, având în vedere că se efectueaza 1000000 operații cu funcția _taketime_. Acesta este rezultatul execuției paralele cu 2 core-uri.

### În exemplul anterior nu am salvat (materializat) niciun rezultat intermediar din `interim`, deci o altă operație simplă (precum numărarea elementelor pozitive) va dura aproape la fel de mult timp.

In [18]:
%%time
print(interim.filter(lambda x:x>0).count())

500000
CPU times: user 93.8 ms, sys: 8.26 ms, total: 102 ms
Wall time: 14 s


## Păstrarea în cache (_caching_) pentru reducerea timpului de calcul pe operații similare (demersul presupune utilizarea memoriei)

### Efectuați calculele anterioare utilizând metoda `cache` pentru a determina graful de dependențe să planifice _caching_-ul

In [19]:
%%time
interim = rdd1.map(lambda x: taketime(x)).cache()

CPU times: user 6.37 ms, sys: 592 µs, total: 6.96 ms
Wall time: 31.4 ms


In [20]:
#Afișați graful de dependențe pentru interim
print(interim.toDebugString().decode())

(2) PythonRDD[4] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289 [Memory Serialized 1x Replicated]


In [21]:
%%time
#Obțineți suma elementelor cu ajutorul metodei reduce și a cache-ului
interim.reduce(lambda x,y:x+y)

CPU times: user 141 ms, sys: 16.9 ms, total: 158 ms
Wall time: 17.7 s


-0.2887054679684353

### Aplicați metoda `filter` pentru numărarea elementelor pozitive. Calculul se va realiza cu ajutorul rezultatului din cache.

In [22]:
%%time
#Aplicați metoda filter
interim.filter(lambda x:x>0).count()

CPU times: user 15.9 ms, sys: 2.24 ms, total: 18.2 ms
Wall time: 1.05 s


500000

#### Se observă că durata execuției a fost mai scurtă în condițiile utilizării rezultatului din _cache_, ce a putut fi folosit pentru compararea cu 0 și numarare.

In [23]:
sc.stop()